In [1]:
import os
os.chdir("..")
import descriptor.three_desc_model as tdm
from descriptor.cutoffs import Polynomial
import numpy as np
import pandas as pd
from scipy.optimize import minimize,fmin
import matplotlib.pyplot as plt
import GPy
from IPython.display import display
dataset = pd.read_csv('turb_data.csv')
dataset= dataset.drop(['ID','turb_num'], axis=1)
symbol="Siemens"
numsims=dataset["IDnum"].iloc[-1]+1
turb = [symbol]
nl=tdm.NeighborlistCalculator(cutoff=4001,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)

In [4]:
def f(x):
    Gs = {"Siemens": [{"type":"G2", "turbine":"Siemens","eta":x[0], "offset": x[3]},
                  {"type":"G4", "elements":"Siemens","eta":x[1], "gamma":x[4]},
                  {"type":"G6", "elements":"Siemens","eta":x[2], "gamma": x[5]}
                 ]}
    finpr=tdm.FingerprintCalculator(cutoff=4001,Gs=Gs,Rct=3000,delta_R=100,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)
    count=0
    dataset_fp=np.empty(shape=(0, 3))
    dataset_ws=np.empty(shape=(0, 1))
    dataset_rws=np.empty(shape=(0, 1))
    dataset_pos=np.empty(shape=(0, 2))
    for i in range(numsims):
        numturb=dataset["Num_tot_turb"].iloc[count]
        position = np.empty((numturb,2))
        rws = np.empty((numturb,1))
        ws = np.empty((numturb,1))
        fp = np.empty((numturb,3))
        for k in range(numturb):
            position[k,0]=dataset.at[count,"X_coord"]
            position[k,1]=dataset.at[count,"Y_coord"]
            rws[k,0]=dataset["ref_wind_speed"].iloc[count]
            ws[k,0]=dataset.at[count,"Ref_wind"]
            count = count+ 1
        neigh=nl.calculate(turb*numturb,position)
        fingerprints=finpr.calculate(turb*numturb,position,neigh,symbol)
        fingerprints=np.array(fingerprints)
        dataset_fp=np.append(dataset_fp,fingerprints,axis=0)
        dataset_rws=np.append(dataset_rws,rws,axis=0)
        dataset_ws=np.append(dataset_ws,ws,axis=0)
        dataset_pos=np.append(dataset_pos,position,axis=0)
    dataset=np.concatenate((dataset_fp, dataset_ws, dataset_rws),axis=1)
    dataset = pd.DataFrame(dataset, columns = ['Fingerprint(G2)','Fingerprint(G4)','Fingerprint(G6)','Wind_Speed','Ref_Wind_Speed'])
    X=dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)","Wind_Speed"]].to_numpy()
    Y=dataset[["Ref_Wind_Speed"]].to_numpy()
    train_dataset = dataset.sample(frac=0.8, random_state=0)
    test_dataset = dataset.drop(train_dataset.index)
    Xtrain=train_dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)","Wind_Speed"]].to_numpy()
    Ytrain=train_dataset[["Ref_Wind_Speed"]].to_numpy()
    Xtest=test_dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)","Wind_Speed"]].to_numpy()
    Ytest=test_dataset[["Ref_Wind_Speed"]].to_numpy()
    # define kernel
    ker = GPy.kern.RBF(3,lengthscale=0.1)# + GPy.kern.White(2)
    # create simple GP model
    m = GPy.models.GPRegression(Xtrain,Ytrain,ker)
    # optimize and plot
    m.optimize(max_f_eval = 1000)

    [Ypred,Yvar]=m.predict(Xtest)
    return np.max(Ypred-Ytest)

In [5]:
x0 = [5.3, 5., 2., 1., 8., 1.]
bounds= ((0,10),(0,10),(0,10),(0,10),(0,10),(0,10))
res = minimize(f, x0, bounds= bounds, options={'disp':True})

UnboundLocalError: local variable 'dataset' referenced before assignment